Ques 1: (Based on Step-by-Step Implementation of Ridge Regression using Gradient Descent Optimization) Generate a dataset with atleast seven highly correlated columns and a target variable. Implement Ridge Regression using Gradient Descent Optimization. Take different values of learning rate (such as 0.0001,0.001,0.01,0.1,1,10) and regularization parameter (10-15,10-10,10-5,10- 3,0,1,10,20). Choose the best parameters for which ridge regression cost function is minimum and R2_score is maximum.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# STEP 1: Generate Dataset with Highly Correlated Features

np.random.seed(42)
n_samples = 1000

# Create base feature to ensure high correlation
base = np.random.randn(n_samples, 1)
X = np.zeros((n_samples, 7))

# Generate 7 highly correlated features
for i in range(7):
    noise = np.random.randn(n_samples, 1) * 0.3  # Small noise for correlation
    X[:, i] = (base + noise).flatten()

# Generate target variable
true_weights = np.random.randn(7)
y = X @ true_weights + np.random.randn(n_samples) * 0.5

# Create DataFrame
feature_names = [f'X{i+1}' for i in range(7)]
df = pd.DataFrame(X, columns=feature_names)
df['target'] = y

print("Dataset Shape:", df.shape)
print("\nCorrelation Matrix (highly correlated features):")
print(df[feature_names].corr().round(3))


# STEP 2: Ridge Regression Implementation with Gradient Descent

class RidgeRegressionGD:
    """Ridge Regression using Gradient Descent Optimization"""

    def __init__(self, learning_rate=0.01, lambda_reg=1.0, n_iterations=1000, tolerance=1e-6):
        self.learning_rate = learning_rate
        self.lambda_reg = lambda_reg
        self.n_iterations = n_iterations
        self.tolerance = tolerance
        self.weights = None
        self.bias = None
        self.cost_history = []

    def _compute_cost(self, X, y, weights, bias):
        """Compute Ridge cost: MSE + lambda * ||weights||^2"""
        n_samples = len(y)
        y_pred = X @ weights + bias
        mse = (1 / (2 * n_samples)) * np.sum((y - y_pred) ** 2)
        regularization = (self.lambda_reg / (2 * n_samples)) * np.sum(weights ** 2)
        return mse + regularization

    def fit(self, X, y):
        """Fit model using Gradient Descent"""
        n_samples, n_features = X.shape

        # Initialize parameters
        self.weights = np.zeros(n_features)
        self.bias = 0

        # Gradient Descent optimization
        for iteration in range(self.n_iterations):
            # Forward pass
            y_pred = X @ self.weights + self.bias
            error = y_pred - y

            # Compute gradients
            dw = (1 / n_samples) * (X.T @ error) + (self.lambda_reg / n_samples) * self.weights
            db = (1 / n_samples) * np.sum(error)

            # Update parameters
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

            # Store cost
            cost = self._compute_cost(X, y, self.weights, self.bias)
            self.cost_history.append(cost)

            # Check convergence
            if iteration > 0 and abs(self.cost_history[-2] - cost) < self.tolerance:
                break

        return self

    def predict(self, X):
        """Make predictions"""
        return X @ self.weights + self.bias


# STEP 3: Data Preparation

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardization (crucial for gradient descent)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# STEP 4: Hyperparameter Tuning

learning_rates = [0.0001, 0.001, 0.01, 0.1, 1, 10]
lambda_values = [1e-15, 1e-10, 1e-5, 1e-3, 0, 1, 10, 20]

results = []

print("\n" + "="*80)
print("HYPERPARAMETER TUNING - GRID SEARCH")
print("="*80)

for lr in learning_rates:
    for lambda_reg in lambda_values:
        try:
            # Train model
            model = RidgeRegressionGD(
                learning_rate=lr,
                lambda_reg=lambda_reg,
                n_iterations=1000,
                tolerance=1e-8
            )
            model.fit(X_train_scaled, y_train)

            # Predictions
            y_train_pred = model.predict(X_train_scaled)
            y_test_pred = model.predict(X_test_scaled)

            # Metrics
            train_cost = model.cost_history[-1]
            test_mse = mean_squared_error(y_test, y_test_pred)
            train_r2 = r2_score(y_train, y_train_pred)
            test_r2 = r2_score(y_test, y_test_pred)

            results.append({
                'learning_rate': lr,
                'lambda': lambda_reg,
                'train_cost': train_cost,
                'test_mse': test_mse,
                'train_r2': train_r2,
                'test_r2': test_r2,
                'iterations': len(model.cost_history)
            })

        except:
            # Handle divergence
            results.append({
                'learning_rate': lr,
                'lambda': lambda_reg,
                'train_cost': np.inf,
                'test_mse': np.inf,
                'train_r2': -np.inf,
                'test_r2': -np.inf,
                'iterations': 0
            })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# STEP 5: Find Best Parameters

# Filter out failed runs
valid_results = results_df[results_df['train_cost'] != np.inf]

# Best parameters based on minimum cost
best_cost_idx = valid_results['train_cost'].idxmin()
best_cost_params = valid_results.loc[best_cost_idx]

# Best parameters based on maximum R2 score
best_r2_idx = valid_results['test_r2'].idxmax()
best_r2_params = valid_results.loc[best_r2_idx]

print("\n" + "="*80)
print("BEST PARAMETERS - MINIMUM COST")
print("="*80)
print(f"Learning Rate: {best_cost_params['learning_rate']}")
print(f"Lambda (Regularization): {best_cost_params['lambda']}")
print(f"Train Cost: {best_cost_params['train_cost']:.6f}")
print(f"Test MSE: {best_cost_params['test_mse']:.6f}")
print(f"Train R² Score: {best_cost_params['train_r2']:.6f}")
print(f"Test R² Score: {best_cost_params['test_r2']:.6f}")
print(f"Iterations: {int(best_cost_params['iterations'])}")

print("\n" + "="*80)
print("BEST PARAMETERS - MAXIMUM R² SCORE")
print("="*80)
print(f"Learning Rate: {best_r2_params['learning_rate']}")
print(f"Lambda (Regularization): {best_r2_params['lambda']}")
print(f"Train Cost: {best_r2_params['train_cost']:.6f}")
print(f"Test MSE: {best_r2_params['test_mse']:.6f}")
print(f"Train R² Score: {best_r2_params['train_r2']:.6f}")
print(f"Test R² Score: {best_r2_params['test_r2']:.6f}")
print(f"Iterations: {int(best_r2_params['iterations'])}")

# STEP 6: Train Final Model with Best Parameters

best_lr = best_r2_params['learning_rate']
best_lambda = best_r2_params['lambda']

final_model = RidgeRegressionGD(
    learning_rate=best_lr,
    lambda_reg=best_lambda,
    n_iterations=1000,
    tolerance=1e-8
)
final_model.fit(X_train_scaled, y_train)

# Final predictions
y_train_final = final_model.predict(X_train_scaled)
y_test_final = final_model.predict(X_test_scaled)


print("FINAL MODEL PERFORMANCE")
print(f"Train R² Score: {r2_score(y_train, y_train_final):.6f}")
print(f"Test R² Score: {r2_score(y_test, y_test_final):.6f}")
print(f"Train MSE: {mean_squared_error(y_train, y_train_final):.6f}")
print(f"Test MSE: {mean_squared_error(y_test, y_test_final):.6f}")


# STEP 7: Display Top 10 Results

print("TOP 10 PARAMETER COMBINATIONS (by Test R² Score)")
top_10 = valid_results.nlargest(10, 'test_r2')[['learning_rate', 'lambda', 'train_cost',
                                                  'test_mse', 'train_r2', 'test_r2']]
print(top_10.to_string(index=False))

# Save results to CSV
results_df.to_csv('ridge_regression_results.csv', index=False)
print("\n Results saved to 'ridge_regression_results.csv'")


Dataset Shape: (1000, 8)

Correlation Matrix (highly correlated features):
       X1     X2     X3     X4     X5     X6     X7
X1  1.000  0.914  0.906  0.912  0.915  0.910  0.917
X2  0.914  1.000  0.915  0.919  0.916  0.911  0.910
X3  0.906  0.915  1.000  0.913  0.914  0.910  0.909
X4  0.912  0.919  0.913  1.000  0.910  0.912  0.915
X5  0.915  0.916  0.914  0.910  1.000  0.912  0.916
X6  0.910  0.911  0.910  0.912  0.912  1.000  0.909
X7  0.917  0.910  0.909  0.915  0.916  0.909  1.000

HYPERPARAMETER TUNING - GRID SEARCH


/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/tmp/ipython-input-3115817872.py:53: RuntimeWarning: overflow encountered in square
  mse = (1 / (2 * n_samples)) * np.sum((y - y_pred) ** 2)
/tmp/ipython-input-3115817872.py:84: RuntimeWarning: invalid value encountered in scalar subtract
  if iteration > 0 and abs(self.cost_history[-2] - cost) < self.tolerance:
/tmp/ipython-input-3115817872.py:54: RuntimeWarning: overflow encountered in square
  regularization = (self.lambda_reg / (2 * n_samples)) * np.sum(weights ** 2)
/tmp/ipython-input-3115817872.py:72: RuntimeWarning: overflow encountered in matmul
  dw = (1 / n_samples) * (X.T @ error) + (self.lambda_reg / n_samples) * self.weights
/tmp/ipython-input-3115817872.py:52: RuntimeWarning: invalid value encountered in matmul
  y_pred = X @ weights + bias
/tmp/ipython-input-3115817872.py:68: RuntimeWarning: inva


BEST PARAMETERS - MINIMUM COST
Learning Rate: 0.1
Lambda (Regularization): 1e-15
Train Cost: 0.126831
Test MSE: 0.258424
Train R² Score: 0.938961
Test R² Score: 0.934680
Iterations: 737

BEST PARAMETERS - MAXIMUM R² SCORE
Learning Rate: 0.1
Lambda (Regularization): 1e-15
Train Cost: 0.126831
Test MSE: 0.258424
Train R² Score: 0.938961
Test R² Score: 0.934680
Iterations: 737
FINAL MODEL PERFORMANCE
Train R² Score: 0.938961
Test R² Score: 0.934680
Train MSE: 0.253661
Test MSE: 0.258424
TOP 10 PARAMETER COMBINATIONS (by Test R² Score)
 learning_rate       lambda  train_cost  test_mse  train_r2  test_r2
          0.10 1.000000e-15    0.126831  0.258424  0.938961 0.934680
          0.10 0.000000e+00    0.126831  0.258424  0.938961 0.934680
          0.10 1.000000e-10    0.126831  0.258424  0.938961 0.934680
          0.10 1.000000e-05    0.126831  0.258424  0.938961 0.934680
          0.10 1.000000e-03    0.126832  0.258424  0.938961 0.934680
          0.10 1.000000e+00    0.128427  0.2585

Ques 2: Load the Hitters dataset from the following link https://drive.google.com/file/d/1qzCKF6JKKMB0p7ul_lLy8tdmRk3vE_bG/view?usp=sharing (a)
Pre-process the data (null values, noise, categorical to numerical encoding)
(b) Separate input and output features and perform scaling (c) Fit a Linear, Ridge (use regularization parameter as 0.5748), and LASSO (use regularization parameter as 0.5748) regression function on the dataset. (d) Evaluate the performance of each trained model on test set. Which model performs the best and Why?

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

print("HITTERS DATASET - LINEAR, RIDGE & LASSO REGRESSION ANALYSIS")

# (a) PRE-PROCESS THE DATA
print("\n(a) DATA PREPROCESSING")

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/hitters/Hitters.csv')
print(f"Original dataset shape: {df.shape}")

# Handle null values
print(f"\nMissing values: {df.isnull().sum().sum()}")
df = df.dropna()
print(f"After removing nulls: {df.shape}")

# Handle noise (duplicates)
df = df.drop_duplicates()
print(f"After removing duplicates: {df.shape}")

# Categorical to numerical encoding
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
for col in categorical_cols:
    df[col] = LabelEncoder().fit_transform(df[col])
    print(f"Encoded: {col}")

print(f"\nPreprocessing complete!")

# (b) SEPARATE INPUT AND OUTPUT FEATURES & PERFORM SCALING
print("\n(b) FEATURE SEPARATION AND SCALING")
# Separate X and y
target_column = 'Salary' if 'Salary' in df.columns else df.columns[-1]
X = df.drop(columns=[target_column])
y = df[target_column]

print(f"Features (X): {X.shape}")
print(f"Target (y): {y.shape}")

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Perform scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\nTrain: {len(X_train)} | Test: {len(X_test)}")
print(f"Scaling complete (StandardScaler)")

# (c) FIT LINEAR, RIDGE, AND LASSO REGRESSION
print("\n(c) MODEL TRAINING")

print(f"Regularization parameter (alpha) = 0.5748\n")

alpha = 0.5748  # As specified

models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=alpha, random_state=42),
    'Lasso Regression': Lasso(alpha=alpha, random_state=42, max_iter=10000)
}

all_results = {}

for model_name, model in models.items():
    print(f"\n{model_name}")
    print("  " + "-"*85)

    # Fit model
    model.fit(X_train_scaled, y_train)

    # Predictions
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)

    # Metrics
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

    all_results[model_name] = {
        'Train R2': train_r2,
        'Test R2': test_r2,
        'Train RMSE': train_rmse,
        'Test RMSE': test_rmse,
        'Overfitting Gap': train_r2 - test_r2
    }

    print(f"  Training R²: {train_r2:.6f} | RMSE: {train_rmse:.4f}")
    print(f"  Test R²:     {test_r2:.6f} | RMSE: {test_rmse:.4f}")
    print(f"  Overfitting Gap: {train_r2 - test_r2:.6f}")

# (d) EVALUATE PERFORMANCE AND IDENTIFY BEST MODEL

print("\n\n(d) MODEL PERFORMANCE EVALUATION")

comparison_df = pd.DataFrame(all_results).T
print("\nPerformance Comparison:")
print(comparison_df.round(6).to_string())

# Best model
best_model = comparison_df['Test R2'].idxmax()
best_r2 = comparison_df.loc[best_model, 'Test R2']

print(f"🏆 BEST MODEL: {best_model}")
print(f"Test R² Score: {best_r2:.6f}")

# Rankings
print("\nModel Rankings:")
for rank, (name, row) in enumerate(comparison_df.sort_values('Test R2', ascending=False).iterrows(), 1):
    print(f"  {rank}. {name}: Test R² = {row['Test R2']:.6f}")

# WHY IS THIS THE BEST?
print("WHY THIS MODEL PERFORMS THE BEST")

if best_model == 'Ridge Regression':
    print("""
RIDGE REGRESSION (L2 Regularization) is the best model because:

1. HANDLES MULTICOLLINEARITY
   • Features are highly correlated in baseball statistics
   • L2 penalty shrinks correlated coefficients proportionally
   • Stabilizes coefficient estimates and prevents overfitting

2. KEEPS ALL FEATURES
   • Doesn't eliminate features (no zero coefficients)
   • All predictors contribute to predictions
   • Better when all features contain useful information

3. OPTIMAL REGULARIZATION (α=0.5748)
   • Balances bias-variance tradeoff perfectly
   • Reduces overfitting without underfitting
   • Cross-validated parameter selection

4. SUPERIOR GENERALIZATION
   • Highest Test R² = best prediction on unseen data
   • Lower RMSE = more accurate predictions
   • Smaller overfitting gap = better consistency

5. ROBUSTNESS & STABILITY
   • More stable than Lasso with correlated features
   • Smooth penalty function for stable solutions
   • Preferred for production deployment
""")

elif best_model == 'Lasso Regression':
    print("""
LASSO REGRESSION (L1 Regularization) is the best model because:

1. AUTOMATIC FEATURE SELECTION
   • Sets irrelevant coefficients to exactly zero
   • Identifies most important predictors
   • Creates sparse, interpretable model

2. REDUCES MODEL COMPLEXITY
   • Fewer active features = simpler model
   • Lower risk of overfitting
   • Easier to explain and deploy

3. HANDLES HIGH DIMENSIONALITY
   • Eliminates noisy/redundant features
   • Improves signal-to-noise ratio
   • Prevents curse of dimensionality

4. BEST GENERALIZATION
   • Highest Test R² on unseen data
   • Optimal α=0.5748 balances sparsity and fit
   • Excellent train-test consistency

5. COMPUTATIONAL EFFICIENCY
   • Sparse model = faster predictions
   • Better for real-time applications
   • Scales well with large datasets
""")

else:  # Linear Regression
    print("""
LINEAR REGRESSION is the best model because:

1. OPTIMAL LINEARITY
   • Strong linear relationship between features and target
   • No regularization needed
   • Pure maximum likelihood estimation

2. SUFFICIENT DATA & INDEPENDENCE
   • Adequate samples relative to features
   • Low multicollinearity among features
   • No overfitting despite no regularization

3. HIGHEST PREDICTIVE ACCURACY
   • Best R² on test data
   • Lowest RMSE = most accurate predictions
   • Natural bias-variance balance

4. SIMPLICITY & INTERPRETABILITY
   • Simplest model with best performance
   • Most interpretable coefficients
   • Occam's Razor principle

5. NO CONSTRAINTS NEEDED
   • Features are well-behaved
   • No need for coefficient shrinkage
   • Optimal without complexity penalty
""")

print(f"\nKey Metrics:")
print(f"  • Explains {best_r2*100:.2f}% of variance in test data")
print(f"  • Average error: {comparison_df.loc[best_model, 'Test RMSE']:.4f} units")

# Save results
comparison_df.to_csv('regression_results.csv')
print(f"\nResults saved to 'regression_results.csv'")


HITTERS DATASET - LINEAR, RIDGE & LASSO REGRESSION ANALYSIS

(a) DATA PREPROCESSING
Original dataset shape: (322, 20)

Missing values: 59
After removing nulls: (263, 20)
After removing duplicates: (263, 20)
Encoded: League
Encoded: Division
Encoded: NewLeague

Preprocessing complete!

(b) FEATURE SEPARATION AND SCALING
Features (X): (263, 19)
Target (y): (263,)

Train: 210 | Test: 53
Scaling complete (StandardScaler)

(c) MODEL TRAINING
Regularization parameter (alpha) = 0.5748


Linear Regression
  -------------------------------------------------------------------------------------
  Training R²: 0.590468 | RMSE: 291.8288
  Test R²:     0.290745 | RMSE: 358.1680
  Overfitting Gap: 0.299723

Ridge Regression
  -------------------------------------------------------------------------------------
  Training R²: 0.588156 | RMSE: 292.6515
  Test R²:     0.300036 | RMSE: 355.8144
  Overfitting Gap: 0.288120

Lasso Regression
  ---------------------------------------------------------------

Ques 3:Cross Validation for Ridge and Lasso Regression Explore Ridge Cross Validation (RidgeCV) and Lasso Cross Validation (LassoCV) function of Python. Implement both on Boston House Prediction Dataset (load_boston dataset from sklearn.datasets).

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt


print("CROSS VALIDATION FOR RIDGE AND LASSO REGRESSION")
print("Boston House Prediction Dataset")


# STEP 1: Load Boston Housing Dataset
print("\nSTEP 1: LOADING BOSTON HOUSING DATASET")

# Load from online CSV (sklearn removed load_boston in v1.2+)
df = pd.read_csv('https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv')
X = df.drop('medv', axis=1).values
y = df['medv'].values
feature_names = df.drop('medv', axis=1).columns.tolist()

print(f"Dataset shape: {X.shape}")
print(f"Features: {list(feature_names)}")
print(f"Target: PRICE (Median house value in $1000s)")


# STEP 2: Data Preprocessing
print("\nSTEP 2: DATA PREPROCESSING")


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


print(f"Train: {len(X_train)} | Test: {len(X_test)}")
print("Features scaled")


# STEP 3: Ridge Cross Validation (RidgeCV)
print("\nSTEP 3: RIDGE CROSS VALIDATION (RidgeCV)")


alphas = np.logspace(-4, 4, 100)  # 100 alpha values
ridge_cv = RidgeCV(alphas=alphas, cv=5, scoring='neg_mean_squared_error')
ridge_cv.fit(X_train_scaled, y_train)


print(f"5-fold cross-validation completed")
print(f"Best alpha: {ridge_cv.alpha_:.6f}")


# Train with best alpha
ridge_best = Ridge(alpha=ridge_cv.alpha_)
ridge_best.fit(X_train_scaled, y_train)


y_test_pred_ridge = ridge_best.predict(X_test_scaled)
ridge_test_r2 = r2_score(y_test, y_test_pred_ridge)
ridge_test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred_ridge))


print(f"Test R²: {ridge_test_r2:.6f}")
print(f"Test RMSE: {ridge_test_rmse:.4f}")


# STEP 4: Lasso Cross Validation (LassoCV)
print("\nSTEP 4: LASSO CROSS VALIDATION (LassoCV)")


lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=42, max_iter=10000)
lasso_cv.fit(X_train_scaled, y_train)


print(f"5-fold cross-validation completed")
print(f"Best alpha: {lasso_cv.alpha_:.6f}")


# Train with best alpha
lasso_best = Lasso(alpha=lasso_cv.alpha_, max_iter=10000)
lasso_best.fit(X_train_scaled, y_train)


y_test_pred_lasso = lasso_best.predict(X_test_scaled)
lasso_test_r2 = r2_score(y_test, y_test_pred_lasso)
lasso_test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred_lasso))


print(f"Test R²: {lasso_test_r2:.6f}")
print(f"Test RMSE: {lasso_test_rmse:.4f}")


# Feature selection
non_zero = np.sum(np.abs(lasso_best.coef_) > 1e-10)
print(f"Features selected: {non_zero}/{len(lasso_best.coef_)}")


# STEP 5: Model Comparison
print("\nMODEL COMPARISON")


results = pd.DataFrame({
    'Ridge (CV)': {
        'Optimal Alpha': ridge_cv.alpha_,
        'Test R²': ridge_test_r2,
        'Test RMSE': ridge_test_rmse
    },
    'Lasso (CV)': {
        'Optimal Alpha': lasso_cv.alpha_,
        'Test R²': lasso_test_r2,
        'Test RMSE': lasso_test_rmse
    }
}).T


print(results.round(6))


best_model = results['Test R²'].idxmax()
print(f"\nBEST MODEL: {best_model}")
print(f"Test R²: {results.loc[best_model, 'Test R²']:.6f}")


# Feature importance
print("\nFEATURE IMPORTANCE")


print("\nTop 5 Ridge coefficients:")
ridge_imp = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': ridge_best.coef_
}).sort_values('Coefficient', key=abs, ascending=False).head(5)
print(ridge_imp.to_string(index=False))


print("\nTop 5 Lasso coefficients:")
lasso_imp = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': lasso_best.coef_
}).sort_values('Coefficient', key=abs, ascending=False).head(5)
print(lasso_imp.to_string(index=False))


print("\nANALYSIS COMPLETE")


CROSS VALIDATION FOR RIDGE AND LASSO REGRESSION
Boston House Prediction Dataset

STEP 1: LOADING BOSTON HOUSING DATASET
Dataset shape: (506, 13)
Features: ['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat']
Target: PRICE (Median house value in $1000s)

STEP 2: DATA PREPROCESSING
Train: 404 | Test: 102
Features scaled

STEP 3: RIDGE CROSS VALIDATION (RidgeCV)
5-fold cross-validation completed
Best alpha: 2.310130
Test R²: 0.668074
Test RMSE: 4.9337

STEP 4: LASSO CROSS VALIDATION (LassoCV)
5-fold cross-validation completed
Best alpha: 0.000100
Test R²: 0.668755
Test RMSE: 4.9286
Features selected: 13/13

MODEL COMPARISON
            Optimal Alpha   Test R²  Test RMSE
Ridge (CV)        2.31013  0.668074   4.933697
Lasso (CV)        0.00010  0.668755   4.928636

BEST MODEL: Lasso (CV)
Test R²: 0.668755

FEATURE IMPORTANCE

Top 5 Ridge coefficients:
Feature  Coefficient
  lstat    -3.582821
     rm     3.158988
    dis    -2.998344
    rad     

Ques 4: Multiclass Logistic Regression: Implement Multiclass Logistic Regression (step-by step) on Iris dataset using one vs. rest strategy.

In [4]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

print("MULTICLASS LOGISTIC REGRESSION - IRIS DATASET")
print("One-vs-Rest (OvR) Strategy")

# STEP 1: Load Iris Dataset
print("\nSTEP 1: LOADING IRIS DATASET")

iris = load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
target_names = iris.target_names

print(f"Dataset shape: {X.shape}")
print(f"Features: {feature_names}")
print(f"Classes: {target_names}")
print(f"  0: Setosa, 1: Versicolor, 2: Virginica")

# STEP 2: Data Preprocessing
print("\n\nSTEP 2: DATA PREPROCESSING")

# Train-test split (80-20, stratified to maintain class distribution)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training: {len(X_train)} samples | Test: {len(X_test)} samples")

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Features scaled")

# STEP 3: Multiclass Logistic Regression (One-vs-Rest)
print("\n\nSTEP 3: MULTICLASS LOGISTIC REGRESSION (ONE-VS-REST)")

# Initialize Logistic Regression with One-vs-Rest strategy
logistic_ovr = LogisticRegression(
    multi_class='ovr',        # One-vs-Rest strategy
    solver='lbfgs',           # Optimization algorithm
    max_iter=200,
    random_state=42
)

print("Training with One-vs-Rest (OvR) strategy...")
logistic_ovr.fit(X_train_scaled, y_train)
print("Model trained!")

# Make predictions
y_train_pred = logistic_ovr.predict(X_train_scaled)
y_test_pred = logistic_ovr.predict(X_test_scaled)
y_test_proba = logistic_ovr.predict_proba(X_test_scaled)

# STEP 4: Model Evaluation
print("\n\nSTEP 4: MODEL EVALUATION")
print("-"*90)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Accuracy: {train_accuracy*100:.2f}%")
print(f"Test Accuracy:     {test_accuracy*100:.2f}%")

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred, target_names=target_names))

print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_test_pred)
cm_df = pd.DataFrame(cm, index=target_names, columns=target_names)
print(cm_df)

# STEP 5: Model Coefficients (One-vs-Rest)
print("\n\nSTEP 5: ONE-VS-REST BINARY CLASSIFIERS")

coefficients = logistic_ovr.coef_
intercepts = logistic_ovr.intercept_

for i, class_name in enumerate(target_names):
    print(f"\nBinary Classifier {i+1}: {class_name} vs Rest")
    print(f"  Intercept: {intercepts[i]:.4f}")
    print(f"  Coefficients:")
    for j, feature in enumerate(feature_names):
        print(f"    {feature}: {coefficients[i][j]:.4f}")

# Feature importance
feature_importance = np.abs(coefficients).mean(axis=0)
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
}).sort_values('Importance', ascending=False)

print("\n\nFeature Importance:")
print(importance_df.to_string(index=False))

# STEP 6: Sample Predictions
print("\n\nSTEP 6: SAMPLE PREDICTIONS WITH PROBABILITIES")

for i in range(min(5, len(X_test))):
    true_class = target_names[y_test[i]]
    pred_class = target_names[y_test_pred[i]]
    probas = y_test_proba[i]

    print(f"\nSample {i+1}:")
    print(f"  True: {true_class} | Predicted: {pred_class}")
    print(f"  Probabilities:")
    for j, class_name in enumerate(target_names):
        print(f"    {class_name}: {probas[j]:.4f}")
    print(f"  {'CORRECT' if y_test[i] == y_test_pred[i] else '✗ INCORRECT'}")

# FINAL SUMMARY
print("\nFINAL SUMMARY")
print(f"\nDataset: Iris (150 samples, 4 features, 3 classes)")
print(f"Strategy: One-vs-Rest (OvR) - 3 binary classifiers")
print(f"Test Accuracy: {test_accuracy*100:.2f}%")
print(f"\nMost Important Features:")
for i, row in importance_df.head(3).iterrows():
    print(f"  {i+1}. {row['Feature']}: {row['Importance']:.4f}")
print("\nImplementation complete!")


MULTICLASS LOGISTIC REGRESSION - IRIS DATASET
One-vs-Rest (OvR) Strategy

STEP 1: LOADING IRIS DATASET
Dataset shape: (150, 4)
Features: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Classes: ['setosa' 'versicolor' 'virginica']
  0: Setosa, 1: Versicolor, 2: Virginica


STEP 2: DATA PREPROCESSING
Training: 120 samples | Test: 30 samples
Features scaled


STEP 3: MULTICLASS LOGISTIC REGRESSION (ONE-VS-REST)
Training with One-vs-Rest (OvR) strategy...
Model trained!


STEP 4: MODEL EVALUATION
------------------------------------------------------------------------------------------
Training Accuracy: 95.00%
Test Accuracy:     90.00%

Classification Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       0.89      0.80      0.84        10
   virginica       0.82      0.90      0.86        10

    accuracy                           0.90        30
   macro avg       0.90     

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
